In [1]:
include("HighDimMixedModels.jl")

Main.HighDimMM

In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
import .HighDimMM   #https://docs.julialang.org/en/v1/manual/variables-and-scoping/

**Set up data and formula, specify factor variables(CategoricalTerm)**

In [3]:
#df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3), d = vcat(repeat([1,2], 4),1))
df = DataFrame(y = rand(rng, 9), a = 1:9, b = rand(rng, 9), c = repeat(["d","e","f"], 3))
f = @formula(y ~ 1 + a + b + c)
contrasts = Dict( :a => ContinuousTerm, :b=>ContinuousTerm, :c => CategoricalTerm)

Dict{Symbol, UnionAll} with 3 entries:
  :a => ContinuousTerm
  :b => ContinuousTerm
  :c => CategoricalTerm

In [4]:
df

,y,a,b,c
,Float64,Int64,Float64,String
1,0.585195,1,0.236782,d
2,0.0773379,2,0.943741,e
3,0.716628,3,0.445671,f
4,0.320357,4,0.763679,d
5,0.653093,5,0.145071,e
6,0.236639,6,0.021124,f
7,0.709684,7,0.152545,d
8,0.557787,8,0.617492,e
9,0.05079,9,0.481531,f


In [5]:
f

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  a(unknown)
  b(unknown)
  c(unknown)

In [33]:
sch = MixedModels.schema(f2,df,contrasts)

StatsModels.Schema with 4 entries:
  c => c
  b => b
  a => a
  y => y


In [34]:
# try for f2
f2 = @formula(y ~ 1 + a + b + (1|c))

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  a(unknown)
  b(unknown)
  (c)->1 | c

In [37]:
form = MixedModels.apply_schema(f2, sch, LinearMixedModel)

FormulaTerm
Response:
  y(continuous)
Predictors:
  1
  a(continuous)
  b(continuous)
  (1 | c)

In [36]:
form = MixedModels.apply_schema(f2, sch, HighDimMM.highDimMixedModel)

FormulaTerm
Response:
  y(continuous)
Predictors:
  1
  a(continuous)
  b(continuous)
  (c)->1 | c

In [20]:
form = MixedModels.apply_schema(f, sch)

FormulaTerm
Response:
  y(continuous)
Predictors:
  1
  a(continuous)
  b(continuous)
  c(DummyCoding:3→2)

FormulaTerm
Response:
  y(unknown)
Predictors:
  1
  a(unknown)
  b(unknown)
  (c)->1 | c

In [38]:
y, pred = MixedModels.modelcols(form, df);

In [39]:
pred

([1.0 1.0 0.236781883208121; 1.0 2.0 0.9437409715735081; … ; 1.0 8.0 0.617492416565387; 1.0 9.0 0.48153065407402607], [1.0 0.0 0.0; 0.0 1.0 0.0; … ; 0.0 1.0 0.0; 0.0 0.0 1.0])

In [44]:
(pred[1])

9×3 Matrix{Float64}:
 1.0  1.0  0.236782
 1.0  2.0  0.943741
 1.0  3.0  0.445671
 1.0  4.0  0.763679
 1.0  5.0  0.145071
 1.0  6.0  0.021124
 1.0  7.0  0.152545
 1.0  8.0  0.617492
 1.0  9.0  0.481531

In [26]:
HMM = highDimMixedModel(f, df, contrasts, 1, 1)

LoadError: UndefVarError: highDimMixedModel not defined

In [11]:
HMM = HighDimMM.highDimMixedModel(f, df, contrasts, 1, 1)

┌ Warning: n >= p in high dimensional matrix
└ @ Main.HighDimMM /Users/zyxu/Documents/julia/highDimMM/bricks.jl:13


Main.HighDimMM.highDimMixedModel{Float64}(y ~ 1 + a + b + c, Main.HighDimMM.highDimMat{Float64, Matrix{Float64}}([1.0; 2.0; … ; 8.0; 9.0]), Main.HighDimMM.XMat{Float64, Matrix{Float64}}([1.0 0.236781883208121; 1.0 0.9437409715735081; … ; 1.0 0.617492416565387; 1.0 0.48153065407402607]), [0.0 0.0; 1.0 0.0; … ; 1.0 0.0; 0.0 1.0], [0.5851946422124186, 0.07733793456911231, 0.7166282400543453, 0.3203570514066232, 0.6530930076222579, 0.2366391513734556, 0.7096838914472361, 0.5577872440804086, 0.05079002172175784])

In [12]:
# Three matrix
HMM.M.M; HMM.X.X; HMM.Z.Z

9×2 Matrix{Float64}:
 0.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  0.0
 1.0  0.0
 0.0  1.0
 0.0  0.0
 1.0  0.0
 0.0  1.0

In [13]:
HMM.M * HMM.X

LoadError: DimensionMismatch("A has dimensions (9,1) but B has dimensions (9,2)")

In [14]:
HMM.M.M * HMM.X.X

LoadError: DimensionMismatch("A has dimensions (9,1) but B has dimensions (9,2)")

In [ ]:
# response
HMM.y

$$
\mathbf{y} \text { is } N_{n}(\mathbf{X} \boldsymbol{\beta}, \mathbf{\Sigma}), \quad \text { where } \quad \mathbf{\Sigma}= \sigma_{z}^{2}
\mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} 
$$

Let $$\mathbf{P}=\mathbf{I}-\mathbf{H}=\mathbf{I}-\mathbf{X}\left(\mathbf{X}^{\prime} \mathbf{X}\right)^{-} \mathbf{X}^{\prime}$$

$$\mathbf{K}=\mathbf{C}\mathbf{P} \in R^{(n-r)\times n}$$

$$
Ky \sim N_{n-r}\left(\mathbf{0}, \mathbf{K} \mathbf{\Sigma} \mathbf{K}^{\prime}\right) = N_{n-r}\left[\mathbf{0}, \mathbf{K}\left(\sigma_{z}^{2} \mathbf{Z} \mathbf{Z}^{\prime}+\sigma^{2} \mathbf{I}_{n} \right) \mathbf{K}^{\prime}\right]
$$

To simplify, we assume fixed effect as full rank, where r is the number of fixed effects.

In [ ]:
A = hcat(HMM.M.M, HMM.X.X)
P = I - A*inv(transpose(A)*A)*transpose(A)
u,s,v = svd(P)
C = transpose(u[:,1:6]) # simplify: assume fixed effect full rank
K = C*P

The objective function is:
$$
\frac{n-r}{2} \ln (2 \pi)-\frac{1}{2} \ln \left|\mathbf{K} \Sigma \mathbf{K}^{\prime}\right|-\frac{1}{2} \mathbf{y}^{\prime} \mathbf{K}^{\prime}\left(\mathbf{K} \Sigma \mathbf{K}^{\prime}\right)^{-1} \mathbf{K} \mathbf{y}
$$